## Cargar bibliotecas

Cargar bibliotecas en Python es crucial para extender las capacidades del lenguaje y aprovechar soluciones especializadas desarrolladas por expertos. Estas bibliotecas ofrecen métodos optimizados y funcionalidades avanzadas que permiten a los desarrolladores resolver problemas complejos de manera eficiente y reutilizar código probado y documentado. Además, importar bibliotecas sigue estándares de la comunidad, asegurando consistencia y facilitando la adopción de mejores prácticas en el desarrollo de software. En conjunto, la importación de bibliotecas en Python no solo mejora la productividad y reduce errores, sino que también amplía la flexibilidad y adaptabilidad del lenguaje para una variedad de aplicaciones y proyectos.

In [ ]:
!pip install soilgrids soiltexture geopandas rasterio rasterstats

In [ ]:
from google.colab import drive
import gdown
import zipfile
import geopandas as gpd
import rasterio
import rasterio.plot
from rasterio.features import shapes
from rasterio.mask import mask
from rasterstats import zonal_stats
from rasterio.transform import from_bounds, from_origin
from soilgrids import SoilGrids
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from rasterio.warp import calculate_default_transform, reproject, Resampling
import requests
from shapely.geometry import shape, mapping
import fiona

## Montar Google Drive

Primero, es necesario conectar tu Google Drive para poder acceder a él desde Google Colab y guardar archivos de trabajo así como los resultados en tu almacenamiento en la nube.

In [ ]:
# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Ruta donde quieres crear la nueva carpeta dentro de tu Google Drive
folder_path = '/content/drive/My Drive/WEAP/'

# Crear la carpeta usando el comando mkdir
!mkdir "{folder_path}"

## Descargar archivos necesarios
El siguiente bloque ejecutará las rutinas necesarias para descargar los archivos esenciales para el desarrollo del taller, que comprenden: i) un modelo digital de elevación en formato raster, y ii) las cuencas delimitadas utilizando las herramientas proporcionadas por el software WEAP en formato vectorial.

In [ ]:
# URL del enlace compartido de Google Drive
gdrive_url = 'https://drive.google.com/uc?id=1HjBliubBs02NAddo1lJz4rA2CgAd0q20'

# Ruta donde descargar el archivo zip en Colab
zip_input_file = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84.zip'

# Descargar el archivo TIF desde Google Drive
gdown.download(gdrive_url, zip_input_file, quiet=False)

# Descomprimir el archivo zip
with zipfile.ZipFile(zip_input_file, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/WEAP/')

# Textura de suelo

## Cargar el shapefile usando GeoPandas

In [ ]:
region = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84_poly.shp'
region = gpd.read_file(region)

## Verificar la proyección del shapefile y transformarla si es necesario

In [ ]:
if region.crs != 'EPSG:4326':
    region = region.to_crs('EPSG:4326')

## SoilGrids: Un sistema global de información del suelo

SoilGrids (SoilGrids Global Soil Information System) es un sistema innovador que proporciona información detallada sobre las propiedades del suelo a escala global. Desarrollado por el Centro de Investigación de Información del Suelo Mundial (ISRIC), SoilGrids utiliza técnicas de aprendizaje automático y modelado estadístico para generar mapas de alta resolución de diversas propiedades del suelo, como:

* Textura del suelo (porcentaje de arena, limo y arcilla)
* Materia orgánica del suelo
* pH del suelo
* Capacidad de intercambio catiónico (CEC)
* Profundidad hasta el lecho rocoso

Estos mapas están disponibles a dos resoluciones espaciales: 250 metros y 1 kilómetro, lo que permite a los usuarios acceder a información del suelo con un nivel de detalle sin precedentes.

SoilGrids se ha convertido en una herramienta invaluable para una amplia gama de aplicaciones, incluyendo:

* Agricultura: Optimizar el manejo del suelo, la selección de cultivos y la aplicación de fertilizantes.
Gestión de recursos hídricos: Evaluar la erosión del suelo, la infiltración del agua y la calidad del agua.
* Cambio climático: Modelar los impactos del cambio climático en el suelo y la agricultura.
Seguridad alimentaria: Evaluar la potencialidad agrícola y la capacidad de producción de alimentos.
* Biodiversidad: Estudiar la relación entre el suelo y la biodiversidad.

La disponibilidad gratuita y abierta de SoilGrids lo convierte en un recurso esencial para investigadores, científicos, profesionales y responsables políticos que trabajan en diversos campos relacionados con el suelo.

### Inicializar SoilGrids

In [ ]:
sg = SoilGrids()

# Obtener los datos de texturas de suelo dentro del área del polígono
# Puedes especificar las profundidades y variables que necesitas
# depths = ['0-5cm', '5-15cm', '15-30cm', '30-60cm', '60-100cm', '100-200cm']
depths = ['0-5cm', '5-15cm', '15-30cm']
variables = ['clay', 'sand', 'silt']

#arcilla: clay
#arena: sand
#limo: silt

# Crear un diccionario para almacenar los datos de texturas
texturas = {var: [] for var in variables}


### Definir las coordenadas y dimensiones del área de interés

In [ ]:
west, south, east, north = -80.324948, -3.874921, -79.970779, -3.416584
width, height = 1034, 891  # Ajusta estos valores según sea necesario
crs = 'urn:ogc:def:crs:EPSG::4326'

### Descargar y procesar datos de suelos
* Función para obtener los datos de una profundidad específica
* Función para combinar datos de diferentes profundidades
* Calcular el promedio ponderado para 0-30 cm

In [ ]:
# Función para obtener los datos de una profundidad específica
def fetch_soilgrids(region, var, depth, output_path):
    coverage_id = f'{var}_{depth}_mean'
    data = sg.get_coverage_data(
        service_id=var,
        coverage_id=coverage_id,
        west=region['west'], south=region['south'], east=region['east'], north=region['north'],
        width=region['width'], height=region['height'],
        crs=region['crs'],
        output=output_path
    )
    return data

# Función para combinar datos de diferentes profundidades
def get_soilgrids_0_30cm(region, var):
    depth_0_5 = fetch_soilgrids(region, var, '0-5cm', f'/content/drive/My Drive/WEAP/{var}_0-5cm.tif')
    depth_5_15 = fetch_soilgrids(region, var, '5-15cm', f'/content/drive/My Drive/WEAP/{var}_5-15cm.tif')
    depth_15_30 = fetch_soilgrids(region, var, '15-30cm', f'/content/drive/My Drive/WEAP/{var}_15-30cm.tif')

    # Leer los datos de los archivos TIFF
    with rasterio.open(f'/content/drive/My Drive/WEAP/{var}_0-5cm.tif') as src:
        data_0_5 = src.read(1)
    with rasterio.open(f'/content/drive/My Drive/WEAP/{var}_5-15cm.tif') as src:
        data_5_15 = src.read(1)
    with rasterio.open(f'/content/drive/My Drive/WEAP/{var}_15-30cm.tif') as src:
        data_15_30 = src.read(1)

    # Calcular el promedio ponderado para 0-30 cm
    combined = (data_0_5 * 5 + data_5_15 * 10 + data_15_30 * 15) / 300
    return combined

# Definir la región de interés
region = {
    'west': west,
    'south': south,
    'east': east,
    'north': north,
    'width': width,
    'height': height,
    'crs': crs
}

# Obtener los datos combinados para las diferentes variables
clay_0_30cm = get_soilgrids_0_30cm(region, 'clay')
silt_0_30cm = get_soilgrids_0_30cm(region, 'silt')
sand_0_30cm = get_soilgrids_0_30cm(region, 'sand')

# Función para guardar los resultados combinados en archivos TIFF
def save_to_tiff(data, output_path, region):
    transform = from_bounds(region['west'], region['south'], region['east'], region['north'], region['width'], region['height'])
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=region['crs'],
        transform=transform,
    ) as dst:
        dst.write(data, 1)

# Guardar los resultados combinados en archivos TIFF
save_to_tiff(clay_0_30cm, '/content/drive/My Drive/WEAP/clay_0_30cm.tif', region)
save_to_tiff(silt_0_30cm, '/content/drive/My Drive/WEAP/silt_0_30cm.tif', region)
save_to_tiff(sand_0_30cm, '/content/drive/My Drive/WEAP/sand_0_30cm.tif', region)

### Función para plotear resultado

In [ ]:
def plot_with_legend(raster_path, title, cmap='viridis', legend_label=''):
    with rasterio.open(raster_path) as src:
        fig, ax = plt.subplots(figsize=(10, 10))

        # Leer y plotear los datos raster
        raster_data = src.read(1)
        img = ax.imshow(raster_data, cmap=cmap, extent=(src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top))

        # Añadir la barra de colores (leyenda)
        cbar = fig.colorbar(img, ax=ax, shrink=0.5)
        cbar.set_label(legend_label)

        # Añadir título y etiquetas de ejes
        ax.set_title(title)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')

        plt.show()

# Ejemplo de uso: plotear el archivo con leyenda
plot_with_legend('/content/drive/My Drive/WEAP/clay_0_30cm.tif',
                 title='Clay Content 0-30 cm',
                 cmap='Oranges',
                 legend_label='Clay Content (%)')


### Función para llenar datos faltantes en un raster y guardar el resultado en otro archivo

In [ ]:
def fill_missing_data_and_save(input_raster_path, output_raster_path):
    with rasterio.open(input_raster_path, 'r+') as src:
        # Leer el raster como una matriz numpy
        raster_data = src.read(1)

        # Encontrar índices de valores faltantes (NaN o NoData)
        missing_mask = np.isnan(raster_data) | (raster_data == src.nodata) | (raster_data == 0)

        # Obtener coordenadas (x, y) de los valores válidos (no faltantes)
        y_valid, x_valid = np.where(~missing_mask)

        # Obtener coordenadas (x, y) de los valores faltantes
        y_missing, x_missing = np.where(missing_mask)

        # Obtener los valores de los píxeles vecinos válidos
        values_valid = raster_data[y_valid, x_valid]

        # Interpolar los valores faltantes basados en los valores vecinos válidos
        interpolated_values = griddata((x_valid, y_valid), values_valid, (x_missing, y_missing), method='linear')

        # Llenar los valores faltantes en el raster con los valores interpolados
        raster_data[y_missing, x_missing] = interpolated_values

        # Guardar los datos llenos en otro archivo raster
        profile = src.profile
        with rasterio.open(output_raster_path, 'w', **profile) as dst:
            dst.write(raster_data, 1)


In [ ]:
# Ejemplo de uso: llenar datos faltantes y guardar el resultado en otro archivo
input_file = '/content/drive/My Drive/WEAP/clay_0_30cm.tif'
output_file = '/content/drive/My Drive/WEAP/clay_0_30cm_filled.tif'
fill_missing_data_and_save(input_file, output_file)

input_file = '/content/drive/My Drive/WEAP/silt_0_30cm.tif'
output_file = '/content/drive/My Drive/WEAP/silt_0_30cm_filled.tif'
fill_missing_data_and_save(input_file, output_file)

input_file = '/content/drive/My Drive/WEAP/sand_0_30cm.tif'
output_file = '/content/drive/My Drive/WEAP/sand_0_30cm_filled.tif'
fill_missing_data_and_save(input_file, output_file)

### Ejemplo de uso: plotear el archivo con leyenda

In [ ]:
plot_with_legend('/content/drive/My Drive/WEAP/clay_0_30cm_filled.tif',
                 title='Clay Content 0-30 cm',
                 cmap='Oranges',
                 legend_label='Clay Content (%)')

## Calcular textura (USDA)

Lectura de archivos raster

In [ ]:
with rasterio.open('/content/drive/My Drive/WEAP/clay_0_30cm_filled.tif') as src:
    clay_raster = src.read(1)
with rasterio.open('/content/drive/My Drive/WEAP/silt_0_30cm_filled.tif') as src:
    silt_raster = src.read(1)
with rasterio.open('/content/drive/My Drive/WEAP/sand_0_30cm_filled.tif') as src:
    sand_raster = src.read(1)

### Función clasificar textura USDA




In [ ]:
def clasificar_textura_USDA(clay, silt, sand):
    """
    Clasifica la textura del suelo según la USDA.

    Parámetros:
        arena (array): Array de NumPy con los porcentajes de arena (sand).
        arcilla (array): Array de NumPy con los porcentajes de arcilla (clay).
        limo (array): Array de NumPy con los porcentajes de limo (silt).

    Devuelve:
        array: Array de NumPy con la clase de textura USDA para cada píxel.
    """
    textura = np.where(
        (sand >= 70) & (clay <= 15), 1,
        np.where(
            (sand >= 50) & (clay <= 20) & (silt <= 10), 2,
            np.where(
                (sand >= 50) & (clay <= 20), 3,
                np.where(
                    (silt >= 80) & (clay <= 15) & (sand <= 15), 4,
                    np.where(
                        (silt >= 50) & (silt <= 80) & (clay <= 30), 5,
                        np.where(
                            (sand >= 27) & (sand <= 50) & (silt >= 27) & (silt <= 50) & (clay <= 30), 6,
                            np.where(
                                (sand >= 20) & (sand <= 35) & (silt <= 20) & (clay >= 27) & (clay <= 40), 7,
                                np.where(
                                    (clay >= 27) & (clay <= 40) & (sand <= 45) & (silt <= 20), 8,
                                    np.where(
                                        (clay >= 27) & (clay <= 40) & (silt >= 40) & (silt <= 60) & (sand <= 20), 9,
                                        np.where(
                                            (clay >= 40) & (clay <= 60) & (silt >= 40) & (silt <= 60) & (sand <= 20), 10,
                                            np.where(
                                                (clay > 40) & (silt <= 40) & (sand <= 40), 11,
                                                np.where(
                                                    (clay >= 27) & (clay <= 40) & (sand >= 20) & (sand <= 60), 12,
                                                    np.nan  # Default case if none of the conditions are met
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    return textura

# 1 = 'Sand'
# 2 = 'Loamy Sand'
# 3 = 'Sandy Loam'
# 4 = 'Silt'
# 5 = 'Silt Loam'
# 6 = 'Loam'
# 7 = 'Sandy Clay Loam'
# 8 = 'Clay Loam'
# 9 = 'Silty Clay Loam'
# 10 = 'Silty Clay'
# 11 = 'Clay'
# 12 = 'Sandy Clay'

In [ ]:
textura_raster = clasificar_textura_USDA(sand_raster, clay_raster, silt_raster)
print(textura_raster)

### Función para crear el archivo raster y escribir los datos

In [ ]:
def obtener_transformacion_y_crs(raster_path):
    # Leer el archivo raster existente
    with rasterio.open(raster_path) as src:
        transform = src.transform
        crs = src.crs
    return transform, crs

def numpy_a_raster(numpy_array, output_path, transform, crs):
    # Definir el perfil del nuevo archivo raster
    perfil = {
        'driver': 'GTiff',
        'height': numpy_array.shape[0],
        'width': numpy_array.shape[1],
        'count': 1,
        'dtype': numpy_array.dtype,
        'crs': crs,
        'transform': transform,
        'nodata': None
    }

    # Crear el archivo raster y escribir los datos
    with rasterio.open(output_path, 'w', **perfil) as dst:
        dst.write(numpy_array, 1)


In [ ]:
# Ruta al archivo raster existente
raster_path = '/content/drive/My Drive/WEAP/clay_0_30cm_filled.tif'

# Obtener la transformación y el CRS del raster existente
transform, crs = obtener_transformacion_y_crs(raster_path)

# Ruta al archivo de salida
output_path = '/content/drive/My Drive/WEAP/USDA_0_30cm.tif'

# Convertir la matriz NumPy en un archivo raster usando la misma transformación y CRS
numpy_a_raster(textura_raster, output_path, transform, crs)

print(f'Raster guardado en {output_path}')

### Plotear mapa

In [ ]:
# Leer el archivo raster resultante
output_path = '/content/drive/My Drive/WEAP/USDA_0_30cm.tif'
with rasterio.open(output_path) as src:
    textura_raster = src.read(1)
    transform = src.transform

# Leer el archivo vectorial (shapefile)
vector_path = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84_poly.shp'
gdf = gpd.read_file(vector_path)

# Definir los valores y sus leyendas correspondientes
valores = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
leyendas = ['Sand', 'Loamy Sand', 'Sandy Loam', 'Silt', 'Silt Loam', 'Loam',
            'Sandy Clay Loam', 'Clay Loam', 'Silty Clay Loam', 'Silty Clay',
            'Clay', 'Sandy Clay']

# Definir los colores para cada valor
colores = ['#f5c242', '#f3da75', '#edd382', '#d9d3b0', '#b5c58f', '#a1bd99',
           '#cb9161', '#8e8e69', '#8e8e5b', '#6f7262', '#5b5b4b', '#99897e']

# Crear el mapa de colores personalizado
cmap = mcolors.ListedColormap(colores[:len(valores)])

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(10, 10))

# Mostrar la imagen raster con el mapa de colores personalizado
rasterio.plot.show(textura_raster, ax=ax, cmap=cmap, vmin=1, vmax=12, transform=transform)

# Añadir la capa vectorial al plot
gdf.plot(ax=ax, facecolor='none', edgecolor='red')

# Crear la leyenda
patches = [mpatches.Patch(color=colores[i], label=leyendas[i]) for i in range(len(valores))]
ax.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Añadir título y etiquetas
ax.set_title('Clasificación de Textura del Suelo según USDA')
ax.set_xlabel('Columna')
ax.set_ylabel('Fila')

# Mostrar la figura
plt.show()


# Uso de suelo

## SENTINEL-2 10M LAND USE/LAND COVER: Un conjunto de datos global de alta resolución para la clasificación de la tierra

SENTINEL-2 10M LAND USE/LAND COVER es un conjunto de datos global que proporciona información detallada sobre el uso y la cobertura del suelo a una resolución espacial de 10 metros. Este conjunto de datos se deriva de imágenes satelitales del sensor SENTINEL-2 de la Agencia Espacial Europea (ESA) y utiliza técnicas de aprendizaje automático de vanguardia para clasificar la tierra en nueve categorías:

* Agua - 1
* Árboles - 2
* Vegetación inundada - 4
* Cultivos - 5
* Área construida - 7
* Suelo desnudo - 8
* Nieve/hielo - 9
* Nubes - 10
* Pastizales 11

Este conjunto de datos de alta resolución es un recurso valioso para una amplia gama de aplicaciones, incluyendo:

Monitoreo del uso del suelo y la cobertura del suelo: SENTINEL-2 10M LAND USE/LAND COVER puede usarse para monitorear cambios en el uso del suelo a lo largo del tiempo, lo que puede ser útil para comprender los impactos del cambio climático, la urbanización y otras actividades humanas.
Gestión de recursos naturales: Este conjunto de datos puede usarse para identificar y mapear áreas de interés para la conservación, la agricultura y la silvicultura.
Planificación urbana y regional: SENTINEL-2 10M LAND USE/LAND COVER puede usarse para informar la planificación del uso del suelo, la gestión del crecimiento urbano y la infraestructura.
Investigación científica: Este conjunto de datos puede usarse para una variedad de investigaciones científicas, como estudios sobre el cambio climático, la biodiversidad y la hidrología.
SENTINEL-2 10M LAND USE/LAND COVER está disponible de forma gratuita y abierta a través de varios repositorios de datos, como

* https://www.arcgis.com/home/item.html?id=cfcb7609de5f478eb7666240902d4d3d
* https://livingatlas.arcgis.com/landcoverexplorer/#mapCenter=39.18600%2C9.04200%2C10&mode=step&timeExtent=2017%2C2022&year=2020&downloadMode=true


### URL del archivo que deseas descargar

In [ ]:
url = 'https://lulctimeseries.blob.core.windows.net/lulctimeseriesv003/lc2021/17M_20210101-20220101.tif'

# Ruta donde deseas guardar el archivo descargado
file_path = '/content/drive/My Drive/WEAP/r17M_20210101-20220101.tif'

# Realizar la solicitud GET
response = requests.get(url, stream=True)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Abrir un archivo en modo binario para escribir los datos descargados
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print("Archivo descargado exitosamente.")
else:
    print(f"Error al descargar el archivo: {response.status_code}")

### Reproyectar a WGS84

In [ ]:
# Ruta al archivo raster de entrada y salida
input_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101.tif'
output_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_wgs84.tif'

# Definir el EPSG de la nueva proyección
target_crs = 'EPSG:4326'

# Abrir el raster de entrada y obtener sus metadatos
with rasterio.open(input_raster) as src:
    # Obtener la transformación predeterminada y las dimensiones del raster reproyectado
    transform, width, height = calculate_default_transform(src.crs, target_crs, src.width, src.height, *src.bounds)

    # Copiar los metadatos del raster de entrada
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Crear un archivo de salida para el raster reproyectado
    with rasterio.open(output_raster, 'w', **kwargs) as dst:
        # Reproyectar el raster
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest  # Puedes ajustar el método de remuestreo según tus necesidades
            )

print("Raster reproyectado exitosamente.")


### Recortar con archivo shapefile (polígono)

In [ ]:
# Rutas de los archivos de entrada y salida
input_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_wgs84.tif'
shapefile = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84_poly.shp'
output_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_wgs84.tif'

# Cargar el shapefile como GeoDataFrame
gdf = gpd.read_file(shapefile)

# Abrir el raster reproyectado
with rasterio.open(input_raster) as src:
    # Recortar el raster usando el polígono del shapefile
    out_image, out_transform = mask(src, gdf.geometry, crop=True)

    # Actualizar la metadata del raster resultante
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # Guardar el raster recortado en un nuevo archivo
    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)

print("Raster recortado exitosamente.")


### Reclasificar raster de cobertura|uso de suelos

In [ ]:
input_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_wgs84.tif'
output_raster = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_reclass_wgs84.tif'

# Definir las reglas de reclasificación en formato de diccionario
# Las claves son los valores originales y los valores son los nuevos valores
reclassification_rules = {
    1: 1, # Agua
    2: 2, # Árboles
    4: 4, # Vegetación inundada
    5: 5, # Cultivos
    7: 7, # Área construida
    8: 8, # Suelo desnudo
    9: 9, # Nieve/hielo
    10:0, # Nubes ----
    11:11 # Pastizales
}

# Abrir el raster de entrada
with rasterio.open(input_raster) as src:
    # Leer los datos del raster en un array numpy
    raster_data = src.read(1)

    # Crear un array vacío para los datos reclasificados
    reclassified_data = np.copy(raster_data)

    # Aplicar las reglas de reclasificación
    for original_value, new_value in reclassification_rules.items():
        reclassified_data[raster_data == original_value] = new_value

    # Actualizar los metadatos del raster
    out_meta = src.meta.copy()

    # Guardar el raster reclasificado en un nuevo archivo
    with rasterio.open(output_raster, 'w', **out_meta) as dest:
        dest.write(reclassified_data, 1)

print("Raster reclasificado exitosamente.")


In [ ]:
# Plotear mapa
# Leer el archivo raster resultante
with rasterio.open(output_path) as src:
    cobertura_raster = src.read(1)
    transform = src.transform

# Leer el archivo vectorial (shapefile)
vector_path = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84_poly.shp'
gdf = gpd.read_file(vector_path)

# Definir los valores y sus leyendas correspondientes
valores = [0, 1, 2, 4, 5, 7, 8, 9, 11]
leyendas = ['NaN', 'Agua', 'Árboles', 'Vegetación inundada', 'Cultivos',
            'Área construida', 'Suelo desnudo', 'Nieve/hielo', 'Pastizales']

# Definir los colores para cada valor
colores = ['#FF0000', '#f3da75', '#edd382', '#d9d3b0', '#b5c58f', '#a1bd99',
           '#cb9161', '#8e8e69', '#8e8e5b']

# Crear el mapa de colores personalizado
cmap = mcolors.ListedColormap(colores[:len(valores)])

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(10, 10))

# Mostrar la imagen raster con el mapa de colores personalizado
rasterio.plot.show(textura_raster, ax=ax, cmap=cmap, vmin=1, vmax=12, transform=transform)

# Añadir la capa vectorial al plot
gdf.plot(ax=ax, facecolor='none', edgecolor='red')

# Crear la leyenda
patches = [mpatches.Patch(color=colores[i], label=leyendas[i]) for i in range(len(valores))]
ax.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Añadir título y etiquetas
ax.set_title('Cobertura | Uso de suelo')
ax.set_xlabel('long')
ax.set_ylabel('lat')

# Mostrar la figura
plt.show()

# Determinación de áreas de uso y tipo de suelo dentro de la cuenca

### Rutas de los archivos

In [ ]:
shapefile_path = '/content/drive/My Drive/WEAP/WEAPCatchment_wgs84_poly.shp'
soil_type_raster_path = '/content/drive/My Drive/WEAP/USDA_0_30cm.tif'
land_use_raster_path = '/content/drive/My Drive/WEAP/r17M_20210101-20220101_reclass_wgs84.tif'
output_csv_path = '/content/drive/My Drive/WEAP/intersection_result.csv'

In [ ]:
# Cargar el shapefile y filtrar solo el atributo 'name'
gdf = gpd.read_file(shapefile_path)[['Name', 'geometry']]

# Función para vectorizar un raster
def vectorize_raster(raster_path, attribute_name):
    with rasterio.open(raster_path) as src:
        image = src.read(1)  # leer la primera banda
        image = image.astype(np.int32)  # convertir a int32
        mask = image != src.nodata
        results = (
            {'properties': {attribute_name: v}, 'geometry': s}
            for i, (s, v) in enumerate(
                shapes(image, mask=mask, transform=src.transform))
        )
        vector_gdf = gpd.GeoDataFrame.from_features(results)
        vector_gdf.crs = src.crs
    return vector_gdf

# Vectorizar los rasters
soil_type_gdf = vectorize_raster(soil_type_raster_path, 'tipo')
land_use_gdf = vectorize_raster(land_use_raster_path, 'uso')

# Diccionarios para reemplazar los valores
uso_mapping = {
    1: 'Water',
    2: 'Trees',
    4: 'Flooded vegetation',
    5: 'Crops',
    7: 'Built Area',
    8: 'Bare ground',
    9: 'Snow/Ice',
    11: 'Rangeland'
}

tipo_mapping = {
    1: 'Sand',
    2: 'Loamy Sand',
    3: 'Sandy Loam',
    4: 'Silt',
    5: 'Silt Loam',
    6: 'Loam',
    7: 'Sandy Clay Loam',
    8: 'Clay Loam',
    9: 'Silty Clay Loam',
    10: 'Silty Clay',
    11: 'Clay',
    12: 'Sandy Clay'
}

# Reemplazar los valores en los GeoDataFrames
soil_type_gdf['tipo'] = soil_type_gdf['tipo'].replace(tipo_mapping)
land_use_gdf['uso'] = land_use_gdf['uso'].replace(uso_mapping)

# Intersectar los tres datasets
intersection_gdf = gpd.overlay(gdf, soil_type_gdf, how='intersection')
intersection_gdf = gpd.overlay(intersection_gdf, land_use_gdf, how='intersection')

# Transformar a EPSG:24877
intersection_gdf = intersection_gdf.to_crs(epsg=24877)

# Calcular el área en hectáreas
intersection_gdf['area_ha'] = intersection_gdf['geometry'].area / 10000

# Seleccionar columnas y guardar en CSV
result_df = intersection_gdf[['Name', 'tipo', 'uso', 'area_ha']]
result_df.to_csv(output_csv_path, index=False)

print("Proceso completado y archivo CSV guardado exitosamente.")
